In [6]:
import requests
import json
import pandas as pd
import glob
import json
from multiprocessing import Pool
import multiprocessing

In [7]:
glob.glob('./data/*.csv')

['./data/태풍링링_comment_info.csv',
 './data/ht_태풍_타파_post_info.csv',
 './data/ht_태풍_링링_comments_info.csv',
 './data/태풍_location_info.csv',
 './data/태풍타파_location_info.csv',
 './data/태풍_post_info.csv',
 './data/ht_태풍_타파_comments_info.csv',
 './data/태풍타파_post_info.csv',
 './data/태풍링링_location.csv',
 './data/ht_태풍_다나스_post_info.csv',
 './data/태풍타파_comment_info.csv',
 './data/ht_태풍_다나스_comments_info.csv',
 './data/태풍_comment_info.csv',
 './data/태풍링링_post_info.csv',
 './data/ht_태풍_링링_post_info.csv',
 './data/ht_태풍_프란시스코_post_info.csv',
 './data/ht_태풍_프란시스코_comments_info.csv']

In [77]:
loc = pd.read_csv('./data/태풍_location_info.csv')
loc_tapa = pd.read_csv('./data/태풍타파_location_info.csv')
loc_ring = pd.read_csv('./data/태풍링링_location.csv')

In [14]:
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?'
header = {'Authorization': 'KakaoAK 개인토큰'}

In [79]:
multiprocessing.cpu_count()

12

In [51]:
def get_address(x):
    try:
        q = {"x":x['Longitude'], "y":x['Latitude']}
        r = requests.get(url, headers=headers, params=q)
        result = json.loads(r.text)
        return result['documents'][0]['region_1depth_name'], result['documents'][0]['region_2depth_name'], result['documents'][0]['region_3depth_name']
    except Exception as e:
        return 'Not Found'

In [82]:
def get_address_multiprocessing(df):
    df['행정구'] = df[['Latitude', 'Longitude']].apply(get_address, axis=1)
    return df

In [80]:
def parallelize_datafram(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [90]:
def preprocessing(df):
    df['행정구_depth1'] = df['행정구'].str[0]
    df['행정구_depth2'] = df['행정구'].str[1]
    df['행정구_depth3'] = df['행정구'].str[2]
    df.loc[df['행정구'] == 'Not Found', '행정구'] = np.nan
    df.loc[df['행정구'].isnull(), '행정구_depth1'] = np.nan
    df.loc[df['행정구'].isnull(), '행정구_depth2'] = np.nan
    df.loc[df['행정구'].isnull(), '행정구_depth3'] = np.nan
    return df

In [88]:
loc = parallelize_datafram(loc, get_address_multiprocessing)
loc_ring = parallelize_datafram(loc_ring, get_address_multiprocessing)
loc_tapa = parallelize_datafram(loc_tapa, get_address_multiprocessing)

In [96]:
loc = preprocessing(loc)
loc_tapa = preprocessing(loc_tapa)
loc_ring = preprocessing(loc_ring)

In [97]:
loc.to_csv('./data/태풍_location_info_행정구.csv')
loc_tapa.to_csv('./data/태풍타파_location_info_행정구.csv')
loc_ring.to_csv('./data/태풍링링_location_행정구.csv')

,Post_created_time,location,GoogleMap,Latitude,Longitude,행정구,행정구_depth1,행정구_depth2,행정구_depth3
0,2019-09-19 00:02:05,전포카페거리 전리단길,"https://maps.google.com/maps?q=35.1553018503,1...",35.155302,129.062662,"(부산광역시, 부산진구, 전포동)",부산광역시,부산진구,전포동
1,2019-09-19 03:26:01,Surfyy Beach : 서피비치,"https://maps.google.com/maps?q=38.0279262008,1...",38.027926,128.717501,"(강원도, 양양군, 현북면)",강원도,양양군,현북면
2,2019-09-19 05:07:04,미소아트,"https://maps.google.com/maps?q=35.2619324583,1...",35.261932,128.864243,"(경상남도, 김해시, 삼계동)",경상남도,김해시,삼계동
3,2019-09-19 08:38:02,전주대학교,"https://maps.google.com/maps?q=35.8160900233,1...",35.816090,127.089043,"(전라북도, 전주시 완산구, 효자동3가)",전라북도,전주시 완산구,효자동3가
4,2019-09-19 08:49:05,Secretgarden JEJU,"https://maps.google.com/maps?q=33.5283020863,1...",33.528302,126.659961,"(제주특별자치도, 제주시, 조천읍)",제주특별자치도,제주시,조천읍
...,...,...,...,...,...,...,...,...,...
3816,2019-10-17 17:30:02,The MVL,"https://maps.google.com/maps?q=34.7424995236,1...",34.742500,127.754591,"(전라남도, 여수시, 수정동)",전라남도,여수시,수정동
3817,2019-10-18 05:53:03,죽동 어딘가,"https://maps.google.com/maps?q=36.3700620505,1...",36.370062,127.339374,"(대전광역시, 유성구, 죽동)",대전광역시,유성구,죽동
3818,2019-10-19 01:25:01,엔터식스 안양점,"https://maps.google.com/maps?q=37.4023286,126....",37.402329,126.921640,"(경기도, 안양시 만안구, 안양동)",경기도,안양시 만안구,안양동
3819,2019-10-19 02:42:04,더미소,"https://maps.google.com/maps?q=37.239407,126.6...",37.239407,126.663579,"(경기도, 화성시, 송산면)",경기도,화성시,송산면
